In [1]:
import numpy as np
import pandas as pd
import os
import random

In [132]:
random.seed(42)
rows = {}
rows['AB'] = [ random.randint(100,700)  for _ in range(1000) ]
rows['H'] = [ random.randint(50,200) for _ in range(1000) ]
rows['2B'] = [ random.randint(0,50) for _ in range(1000) ]
rows['3B'] = [ random.randint(0,20) for _ in range(1000) ]
rows['HR'] = [ random.randint(0,60) for _ in range(1000) ]
rows['RBI'] = [ random.randint(0,150) for _ in range(1000) ]
rows['SB'] = [ random.randint(1,45) for _ in range(1000) ]
rows['BB'] = [ random.randint(1,150) for _ in range(1000) ]
rows['SO'] = [ random.randint(1,200) for _ in range(1000) ]
rows['HBP'] = [ random.randint(0,30) for _ in range(1000) ]
rows['SF'] = [ random.randint(0,15) for _ in range(1000) ]
dataset = pd.DataFrame(rows)
dataset['BA'] = dataset['H'] / dataset['AB']
dataset['OBP'] = ( dataset['H'] + dataset['BB'] + dataset['HBP'] ) / ( dataset['AB'] + dataset['BB'] + dataset['HBP'] + dataset['SF'] )
dataset['SLG'] = ( dataset['H'] + 2 * dataset['2B'] + 3 * dataset['3B'] + 4 * dataset['HR']                  ) / dataset['AB']
dataset['OPS'] = dataset['OBP'] + dataset['SLG']
dataset.drop(dataset[dataset['OPS'] >= 2].index, inplace=True) # drop outliers

In [133]:
stats = dataset.describe()
stats

,AB,H,2B,3B,HR,RBI,SB,BB,SO,HBP,SF,BA,OBP,SLG,OPS
count,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000
mean,466.852185,122.209512,25.323907,9.785347,29.910026,75.350900,23.110540,76.406170,101.543702,14.973008,7.467866,0.285066,0.396683,0.745776,1.142459
std,139.602248,42.605503,14.461166,6.074733,17.698668,43.618198,13.056677,44.283189,57.899738,8.757305,4.551393,0.127623,0.129930,0.268640,0.367045
min,107.000000,50.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.077508,0.104651,0.156069,0.352306
25%,354.250000,83.000000,13.000000,4.000000,14.000000,39.250000,12.000000,37.000000,52.000000,8.000000,4.000000,0.190154,0.303928,0.541140,0.865227
50%,470.000000,124.500000,25.000000,10.000000,30.000000,72.500000,23.000000,78.000000,102.000000,15.000000,7.000000,0.266692,0.380914,0.702838,1.085604
75%,584.000000,158.000000,37.000000,15.000000,45.000000,113.000000,35.000000,116.000000,152.000000,22.000000,11.000000,0.362376,0.476358,0.929648,1.405206
max,699.000000,200.000000,50.000000,20.000000,60.000000,150.000000,45.000000,150.000000,200.000000,30.000000,15.000000,0.887255,0.915452,1.528736,1.990547


In [134]:
ops_75 = stats['OPS']['75%']
ops_75

1.4052062616844379

In [135]:
ops_50 = stats['OPS']['50%']
ops_50

1.0856040823249344

In [136]:
ops_25 = stats['OPS']['25%']
ops_25

0.8652273985450759

In [137]:
def Evaluate(ops):
    if ops >= ops_75 :
        return 'A'
    elif ops >= ops_50 and ops < ops_75:
        return 'B'
    else:
        return 'C'


In [138]:
dataset['Rank'] = dataset['OPS'].apply(func=Evaluate)

In [139]:
dataset.head()

,AB,H,2B,3B,HR,RBI,SB,BB,SO,HBP,SF,BA,OBP,SLG,OPS,Rank
2,381,75,38,10,9,82,21,101,108,1,4,0.196850,0.363450,0.569554,0.933003,C
3,350,75,32,13,7,29,1,46,163,25,9,0.214286,0.339535,0.588571,0.928106,C
4,328,163,8,3,60,90,11,115,92,20,13,0.496951,0.626050,1.304878,1.930928,A
5,242,92,20,0,48,143,18,92,13,9,14,0.380165,0.540616,1.338843,1.879459,A
7,658,57,20,8,11,149,8,86,75,25,6,0.086626,0.216774,0.250760,0.467534,C


In [140]:
dataset.drop(['OBP', 'SLG', 'OPS', 'BA', 'SB', 'SO', 'RBI'], axis=1, inplace=True)

In [141]:
dataset.head()

,AB,H,2B,3B,HR,BB,HBP,SF,Rank
2,381,75,38,10,9,101,1,4,C
3,350,75,32,13,7,46,25,9,C
4,328,163,8,3,60,115,20,13,A
5,242,92,20,0,48,92,9,14,A
7,658,57,20,8,11,86,25,6,C


In [142]:
X = dataset.drop(['Rank'], axis=1)
y = dataset['Rank']
print(X.head())
print(len(X))

    AB    H  2B  3B  HR   BB  HBP  SF
2  381   75  38  10   9  101    1   4
3  350   75  32  13   7   46   25   9
4  328  163   8   3  60  115   20  13
5  242   92  20   0  48   92    9  14
7  658   57  20   8  11   86   25   6
778


In [143]:
X_features = X.columns
X_features

Index(['AB', 'H', '2B', '3B', 'HR', 'BB', 'HBP', 'SF'], dtype='object')

In [144]:
y.head()

2    C
3    C
4    A
5    A
7    C
Name: Rank, dtype: object

In [145]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [146]:
X = X.to_numpy()
y = y.to_numpy()
print(X.shape)
print(y.shape)

(778, 8)
(778,)


In [153]:
tree_clf = DecisionTreeClassifier(random_state=42, max_depth=4)

In [154]:
f1_scores = cross_val_score(tree_clf, X, y, cv=10, scoring='f1_micro') 

In [155]:
f1_scores

array([0.69620253, 0.79746835, 0.81012658, 0.70886076, 0.70512821,
       0.63636364, 0.75324675, 0.83116883, 0.68831169, 0.84210526])

In [156]:
f1_scores.mean()

0.746898260522511

In [157]:
tree_clf.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [158]:
from sklearn.tree import export_graphviz
import graphviz
dot_data = export_graphviz(tree_clf, out_file='Batting', feature_names=X_features, class_names=['A', 'B', 'C', 'D'])
